In [1]:
%run header.ipynb

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
Using TensorFlow backend.


In [2]:
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

adv_pattern = re.compile('\d+\s?(р(\W|$)|руб|rub|\$|тнг|тенге|грн|гривен)|прода[её]тся|прода[мю]', flags=re.UNICODE | re.IGNORECASE)
assert adv_pattern.search('Цена 1000 тнг.')
assert adv_pattern.search('2500 руб')
assert adv_pattern.search('800 РУБЛЕЙ! ')
assert adv_pattern.search('2700р 22')
assert not adv_pattern.search('2700 рййй')
assert adv_pattern.search('Продам зил-камаз 3500р')
assert adv_pattern.search('Продаю новый дом в Крыму!!!!')
assert adv_pattern.search('Продается дом в р-не')
assert adv_pattern.search('Продаётся квартира')
    
def is_adv(string):
    return 1 if adv_pattern.search(string) else 0

phone_pattern = re.compile('(^|\W)(\+7|8)(-?(\s?\()?(\)\s?)?\s?\d){10}(\W|$)', flags=re.UNICODE | re.IGNORECASE)
assert phone_pattern.search(' тел.89144547633')
#assert phone_pattern.search('Обращаться по тел. 0554106990.') # TODO
assert phone_pattern.search('8-913-03-555-99')
assert phone_pattern.search('8-924-500-8701')
assert phone_pattern.search('89304071675')
assert phone_pattern.search('8-908-659-50-35')
assert phone_pattern.search('+7 (3842) 36-99-83')
assert phone_pattern.search('+79787824200')
assert phone_pattern.search('8-918-090-73-28')
assert phone_pattern.search('89098589641')
assert phone_pattern.search('89098589641.')
assert phone_pattern.search('89098589641 ')
assert phone_pattern.search('т.89098589641')
assert phone_pattern.search('89024058769')
assert phone_pattern.search('т. 89881370418')
assert phone_pattern.search('звонить по телефону 89004176707')
assert phone_pattern.search('8-777-571-70-54')
assert not phone_pattern.search('89024058769777')
assert not phone_pattern.search('8889024058769')
#assert phone_pattern.search('095-247-04-64') # TODO

def has_phone(string):
    return 1 if phone_pattern.search(string) else 0

recipe_pattern = re.compile('\d+\s?(мл|гр|ч\.\s?л|ст\.\s?л|стакан|стакана|грамм)\W|ингредиенты|рецепт', flags=re.UNICODE | re.IGNORECASE)
assert recipe_pattern.search('100 мл \n')
assert recipe_pattern.search('0,5 ч. л\n')
assert recipe_pattern.search('1 ст.л\n')
assert recipe_pattern.search('слоёное тесто 400 гр., ')
assert recipe_pattern.search('5 стакана ')
assert recipe_pattern.search('5 грамм ')

def is_recipe(string):
    return 1 if recipe_pattern.search(string) else 0

def remove_emoji(string):
    return emoji_pattern.sub(r'', string) # e.g: 😊😉😉👇ヅジ

def create_text_features0(texts, result, doc2vec):
    result['embedding'] = texts.preprocessed.apply(doc2vec.infer_vector)

def qwe(x):
    return x.preprocessed.apply(doc2vec.infer_vector)
    
def create_text_features0_batched(texts, doc2vec):
    cpus = int(cpu_count() / 2)
    batch_size = int(texts.shape[0] / cpus + 1)
    batches = [texts.loc[x:(x + batch_size - 1),] for x in range(0, texts.shape[0], batch_size)]
    print('Batches count: %d' % len(batches))
    with Pool(cpus) as p:
        ret = p.map(qwe, batches)
    return pd.concat(ret)
    
def qwe2(x, doc2vec):
    return x.preprocessed.apply(doc2vec.infer_vector)
    
def create_text_features0_batched2(texts, doc2vec):
    batch_size = 5
    batches = [(texts.loc[x:(x + batch_size - 1),], doc2vec) for x in range(0, texts.shape[0], batch_size)]
    print('Batches count: %d' % len(batches))
    with Pool(int(cpu_count() / 2)) as p:
        ret = p.map(qwe2, batches)
    return pd.concat(ret)

    
def create_text_features1(texts, result):
    result['objectId'] = texts.objectId
    result['lang'] = texts.lang
    result['len'] = texts.text.apply(len)
    result['p_len'] = texts.preprocessed.apply(len)
    result['q_count'] = texts.text.str.count('\?')
#        'sentances_count': texts.text.apply(lambda s: s.count('?'))
    result['upper_count'] = texts.text.apply(lambda s: np.char.isupper(list(s)).sum())
    result['emojis_count'] = texts.text.apply(lambda s: len(s) - len(remove_emoji(s)))
    
def create_text_features2(texts, result):
    result['youtube_count'] = texts.text.str.count('youtu\.be|youtube\.com')
    result['links_count'] = texts.text.str.count('http')
    result['ok_videos_count'] = texts.text.str.count('ok\.ru/video')
    result['ok_groups_count'] = texts.text.str.count('ok\.ru/group')
    
def create_text_features3(texts, result):
    result['hashes_count'] = texts.text.str.count('#')
    result['quotes_count'] = texts.text.str.count('«')
    result['mdots_count'] = texts.text.str.count('\.\.')
    result['brackets_balance'] = (texts.text.str.count('\(') - texts.text.str.count('\)')).abs()
    result['is_adv'] = texts.text.apply(is_adv)
    result['is_recipe'] = texts.text.apply(is_recipe)
    
def create_text_features4(texts, result):
    result['has_phone'] = texts.text.apply(has_phone)
    result['is_poll'] = texts.text.str.endswith('Опросы')
    result['e_count'] = texts.text.str.count('!')
    result['lines_count'] = texts.text.str.count('\n')
    result['e_end'] = texts.text.apply(lambda x: x.rstrip().endswith('!'))
    result['q_end'] = texts.text.apply(lambda x: x.rstrip().endswith('?'))
    result['d_end'] = texts.text.apply(lambda x: x.rstrip().endswith('.'))
    
def create_text_features5(texts, result):
    result['md_count'] = texts.text.str.count('январ|феврал|март|апрел|май|июн|июл|август|сентябр|октябр|ноябр|декабр' + 
                                  '|понеденьник|вторник|среда|среду|четверг|пятниц|суббота|субботу|воскресе')
    
    
def create_text_features(texts, doc2vec):
    result = pd.DataFrame({})
    print('create_text_features 0...')
    create_text_features0(texts, result, doc2vec)
    print('create_text_features 1...')
    create_text_features1(texts, result)
    print('create_text_features 2...')
    create_text_features2(texts, result)
    print('create_text_features 3...')
    create_text_features3(texts, result)
    print('create_text_features 4...')
    create_text_features4(texts, result)
    print('create_text_features 5...')
    create_text_features5(texts, result)
    return result

In [4]:
test_texts = parquet.read_table(input_path + '/texts/textsTest/').to_pandas()
train_texts = parquet.read_table(input_path + '/texts/textsTrain').to_pandas()

/usr/local/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
train_texts_shape_0 = parquet.read_table(input_path + '/texts/textsTrain').to_pandas().shape[0]

/usr/local/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
train_texts_shape_0 = 3410916

In [5]:
train_texts_shape_0 = train_texts.shape[0]

In [6]:
%%time
vectorizer = TfidfVectorizer(ngram_range=(1,1), min_df=2)
matrix = vectorizer.fit_transform([' '.join(txt) for txt in train_texts.preprocessed] + 
                                  [' '.join(txt) for txt in test_texts.preprocessed]).astype(np.float32)
matrix

CPU times: user 4min 36s, sys: 55.8 s, total: 5min 32s
Wall time: 6min 17s


In [7]:
del train_texts
del test_texts
gc.collect()

41

In [11]:
del feats
gc.collect()

20

In [8]:
sparse.save_npz(output_path + "/sparse_1gram.npz", matrix)

In [4]:
matrix = sparse.load_npz(output_path + "/sparse_1gram.npz")

In [5]:
%%time
svder = TruncatedSVD(n_components=120, n_iter=40)
feats = svder.fit_transform(matrix)
pd.DataFrame(feats[0:5])

CPU times: user 39min 53s, sys: 4min 7s, total: 44min 1s
Wall time: 37min 32s


In [6]:
pd.DataFrame({'embeddings': list(feats[0:train_texts_shape_0])}).to_pickle(output_path + '/train_120_1_svd')
pd.DataFrame({'embeddings': list(feats[train_texts_shape_0:])}).to_pickle(output_path + '/test_120_1_svd')

In [10]:
feats.dtype

dtype('float32')

In [10]:
doc2vec = Doc2Vec([TaggedDocument(lines,'tag') for lines in test_texts.preprocessed] + 
                  [TaggedDocument(lines,'tag') for lines in train_texts.preprocessed],
                dm=0, vector_size=100, window=5, min_count=2, workers=8)
doc2vec.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
doc2vec.save(output_path + '/doc2vec_all_0_100_5_2')
#doc2vec = Doc2Vec.load(output_path + '/doc2vec_all_23_5_2')

In [11]:
train_texts_features_embedding = train_texts.preprocessed.apply(doc2vec.infer_vector)
test_texts_features_embedding = test_texts.preprocessed.apply(doc2vec.infer_vector)

In [12]:
kclusterer = KMeansClusterer(3, distance=nltk.cluster.util.cosine_distance, repeats=15)
assigned_clusters3_15 = kclusterer.cluster(
    np.concatenate((train_texts_features_embedding, test_texts_features_embedding)), assign_clusters=True)
print('1')

1


In [15]:
pd.DataFrame({'assigned_clusters3_15': assigned_clusters3_15}).to_pickle(output_path + '/assigned_clusters3_15')

In [16]:
pd.DataFrame({'embeddings': train_texts_features_embedding}).to_pickle(output_path + '/train_100_embs')
pd.DataFrame({'embeddings': test_texts_features_embedding}).to_pickle(output_path + '/test_100_embs')

In [13]:
kclusterer = KMeansClusterer(6, distance=nltk.cluster.util.cosine_distance, repeats=15)
assigned_clusters6_15 = kclusterer.cluster(
    np.concatenate((train_texts_features_embedding, test_texts_features_embedding)), assign_clusters=True)
print('1')

KeyboardInterrupt: 

In [20]:
if False:
    train_texts_features = pd.read_pickle(output_path + '/train_text_features2')
    train_texts_features['embedding'] = train_texts.preprocessed.apply(doc2vec.infer_vector)
    train_texts_features.to_pickle(output_path + '/train_text_features3')
    test_texts_features = pd.read_pickle(output_path + '/test_text_features2')
    test_texts_features['embedding'] = test_texts.preprocessed.apply(doc2vec.infer_vector)
    test_texts_features.to_pickle(output_path + '/test_text_features3')
if True:
    test_texts_features = pd.read_pickle(output_path + '/test_text_features7')
    create_text_features5(test_texts, test_texts_features)
    test_texts_features.to_pickle(output_path + '/test_text_features8')
    train_texts_features = pd.read_pickle(output_path + '/train_text_features7')
    create_text_features5(train_texts, train_texts_features)
    train_texts_features.to_pickle(output_path + '/train_text_features8')

In [70]:
%%time
train_text_features = create_text_features(train_texts, doc2vec)

create_text_features 0...
create_text_features 1...
create_text_features 2...
create_text_features 3...


In [71]:
%%time
train_text_features.to_pickle(output_path + '/train_text_features')

CPU times: user 22.5 s, sys: 12.4 s, total: 34.9 s
Wall time: 42.1 s


In [73]:
del train_text_features
del train_texts

In [75]:
%%time
test_text_features = create_text_features(test_texts, doc2vec)

create_text_features 0...
create_text_features 1...
create_text_features 2...
create_text_features 3...
CPU times: user 11min 34s, sys: 8.25 s, total: 11min 42s
Wall time: 11min 47s


In [76]:
%%time
test_text_features.to_pickle(output_path + '/test_text_features')

CPU times: user 3.67 s, sys: 394 ms, total: 4.06 s
Wall time: 4.07 s


In [11]:
def qwe(x):
    x = x.rstrip()
    return x.endswith('.') or x.endswith('?') or x.endswith('?')

In [13]:
test_texts.text.apply(qwe).mean()

0.18356570857106072

In [15]:
test_texts.text.apply(lambda x: x.rstrip().endswith('!')).mean()

0.08159507273671808

In [5]:
test_texts

,objectId,lang,text,preprocessed
0,517288,ru,Квартирник НТВ у Маргулиса_ группа Пилот,"[квартирник, нтв, маргулис, групп, пилот]"
1,9501964,ru,`РОДИТЕЛЯМ`- ОЧЕНЬ ТРОГАТЕЛЬНАЯ ПЕСНЯ!!! Артур...,"[родител, очен, трогательн, песн, артур, халат]"
2,23007371,ru,Сухой Суперджет-100 откажется от западных комп...,"[сух, суперджет, откажет, западн, комплект, те..."
3,38353886,Unknown,"Сгорел сарай, гори и хата","[сгорел, сара, гор, хат]"
4,21192138,ru,Живодёры отрубили лапы собаке 355576121950 ok....,"[живодёр, отруб, лап, собак]"
5,26415073,ru,ok.ru/group/53061241012348 ok.ru/interessnosti...,"[ажурн, маков, кулич, так, нежн, воздушн, мяки..."
6,36734526,ru,ok.ru/group58228491681845 ok.ru/group/58228491...,"[друг, уснул, бар, теб, нужн, тащ, дом, очен, ..."
7,8699823,ru,"Никифоров день\nДата в 2018 году:\t\t26 марта,...","[никифор, ден, дат, год, март, понедельник, др..."
8,12236843,Unknown,ok.ru/group/52742023348461 ok.ru/group52742023...,[]
9,38393782,ru,"ДНР и ЛНР, новости: Войска берут «в кольцо» в ...","[днр, лнр, новост, войск, берут, кольц, донбас..."


In [11]:
for i in range(50400, 55000):
    t = test_texts.loc[i,:].text
    tags = []
    if has_phone(t):
        tags.append('phone')
    if is_adv(t):
        tags.append('adv')
    if is_recipe(t):
        tags.append('recipe')
    print('\n\n---------------------- %d    %s ----------\n\n' % (i, ' '.join(tags)))
    print(t)



---------------------- 50400     ----------


http://dushadevushki.me/2017/11/30/privychka-spat-v-obnimku/?utm_source=contentmoney&utm_medium=99Dt20qc8eBC http://dushadevushki.me/2017/11/30/privychka-spat-v-obnimku/ Если у вас привычка спать отдельно, то вам стоит пересмотреть свои привычки. Спать в обнимку и голышом может принести большую пользу Насколько полезна привычка спать в обнимку?


---------------------- 50401     ----------


Творческая мастерская "АНА". Цветы из шелка. Ручная работа. #цветы #цветыизшелка #цветыручнаяработа #ручнаяработа


---------------------- 50402    adv ----------


Блуза «ШАНЕЛЬ»
Цвет: бордо
Размеры: 50-64
Цена:   1700 р.

Юбка «ЛЕЙЛА»
Цвет: чёрный
Размеры: 46-60
Цена:   1650 р.


---------------------- 50403     ----------


Брехня, конечно, но кто знает...
 
Конец восьмидесятых, нормализация советско-китайских отношений.
На переговорах делегаций министерств обороны двух стран советский генерал предлагает не возвращаться больше к теме трагических со

Возможность дается тем, кто мечтает... Чудо дается тем, кто верит...


---------------------- 50926     ----------


«***Добрый вечер!Любви и счастья!***»http://www.playcast.ru/view/11202976/7a7d1439f0732d5542d775aee2bfb3cc62022217pl


---------------------- 50927     ----------


ДЛЯ МЯСА: красный, черный, душистый перец или гвоздика, майоран, тимьян, тмин, куркума, лук, орегано.

ДЛЯ ПТИЦЫ: тимьян, майоран, розмарин, шалфей, чабрец, базилик.

ДЛЯ РЫБЫ: лавровый лист, белый перец, имбирь, душистый перец, лук, кориандр, перец чили, горчица, укроп, тимьян.

ДЛЯ ГРИЛЯ: красный перец, душистый перец, кардамон, тимьян, майоран, мускатный орех и мускатный цвет, тмин, имбирь, перец чили.

ДЛЯ ДИЧИ: тимьян, душица обыкновенная, душистый перец, красный перец, можжевельник.

ДЛЯ РАГУ: красный перец, имбирь, куркума, кориандр, горчица, кардамон, тмин, черный перец, душистый перец, мускатный орех, гвоздика.

ДЛЯ КАПУСТЫ: кориандр, фенхель, кумин, семя черной горчицы.

ДЛЯ КАРТОФЕЛЯ: кориандр, кур


Пинетки-башмачки
Описание: http://pinetki-bashmachki.vjagu.ru/23/188/6718/


---------------------- 51262     ----------


https://item.taobao.com/item.htm?spm=2013.1.0.0.516534d5OYWwyx&id=563960878477&scm=1007.12144.81309.42296_42296&pvid=fa255e02-08d8-4892-b3cb-adb845526ad5&utparam=%7B%22x_object_type%22%3A%22item%22%2C%22x_object_id%22%3A563960878477%7D&utparam=%7B%22x_obj


---------------------- 51263     ----------


Олег Газманов, зачем ты отправил ребят в АТО? RSA-R7kYXr4 ok.ru/video/1291089482013


---------------------- 51264     ----------


Невозможно сдержать улыбку )


---------------------- 51265     ----------


Havas qildim piyolaga. Choynak unga boshin egar. Lablari uning doim. Go'zal lablarga tegar.


---------------------- 51266     ----------


Հումորային,նոր սիթքոմ ամենալավ դերասանների մասնակցությամբ..Դիտեք՝ http://zarmanq.com/archives/5716


---------------------- 51267     ----------


Трагедия в Кемерово. Зимняя Вишня. #ЖИТЬ 410768378606 ok.ru/video/6302150847


СССР зима 1985. Свадьба ￼


---------------------- 51655     ----------


Игра в футбол тачками http://cdn-ok.com/embed?id=160530533&format=1&sig=b86e5ec668ae6af8&rand=1522421290 https://cdn-ok.com/embed?id=160530533&format=1&sig=b86e5ec668ae6af8&rand=1522421290


---------------------- 51656     ----------


Альба • 10 мест, ЗАПРЕЩЕННЫХ к ПОСЕЩЕНИЮ http://out.pladform.ru/player?pl=18079&type=html5&videoid=101978500 ok.ru/video/242340664381


---------------------- 51657    recipe ----------


БАЛЬЗАМ СВЯЩЕННИКА
 Однажды в поезде моим соседом оказался священник. Он увидел, что я пью сердечные капли, расспросил о болезнях и порекомендовал рецепт бальзама.
Надо взять:
- веточки малины (первого года), вишни, черемухи
- по столовой ложке ягод боярышника и хмеля, 
залить все это холодной кипяченой водой и поставить на огонь. Священник сказал, что пока вода не закипит, следует читать ,Отче наш,   
После закипания отвар полагается держать на медленном огне 10 минут под плотно закрытой крышко

Звоните 80256437768 (Вайбер и т.д.). В наличии и под заказ все размеры!!!Возможна доставка по РБ в течении 3-5 дней!!! Опт и розница.Возможна оплата по карте Халва.Цена 80 рублей.


---------------------- 52011     ----------


Июль, жарко, гроза всю ночь. Закрываю бар, уже четыре утра где-то, достал ключи — собираюсь на сигналку поставить. И тут забегает чувак - одетый только в черный халат с капюшоном на голое тело, в банных тапочках и кожаных перчатках. Говорит, что ему надо спрятаться, просит закрыть дверь. Я понял только, что за ним кто-то гонится, и закрыл жалюзи. Налил ему виски, чтобы он отдышаться. Я угадал, верно, с напитком, потому как он сразу же рассказал мне свою историю. Итак, пару дней назад он познакомился красивой девушкой — в салоне красоты, куда он как курьер доставлял косметику. Они обменялись телефонами, хотя он ничего и не ждал особо — ведь он простой курьер, а она явно богатая красавица. На следующий день она позвонила сама, и уже вечером они катались на ее кабр


Родители отводили детей в дет.сад и не знали,что там происходит!￼￼ 
Пока в садик не пришла новая нянечка..￼. продолжение в группе https://ok.ru/group55349135605783


---------------------- 52426     ----------


Лицензионные Мужские, Женские ароматы Голландского производства. Доступная цена! Идентичность оригиналу! Стойкость 6-8 часов! Подробнее в интернет магазине http://natalina.in.ua/shop/elitnaja_parfjumerija/index.html


---------------------- 52427     ----------


Спасибо за вашу веру и поддержку! 403525405258 ok.ru/video/638974888522


---------------------- 52428     ----------


МУРАЕВ. ВЫДВОРЕНИЕ РОССИЙСКИХ ДИПЛОМАТОВ, ЭТО ФЛЕШМОБ И ПОРОШЕНКО ХОЧЕТ НРАВИТСЯ. ПОНОМАРЕВ. Ссылка на видео. https://youtu.be/1Tqm7lj6AOY


---------------------- 52429     ----------


В Алексеевке инспекторы ДПС вернули родителям потерявшегося ребенка Вечером инспекторы ДПС Александр Малеев и Евгений Фисенко заметили на одной из улиц города Алексеевка гуляющего в одиночестве мальчика. Пообщавшись 


Альба • 10 СУМАСШЕДШИХ, которые ЗАПОСТИЛИ ШОКИРУЮЩИЕ вещи в СОЦСЕТЯХ http://out.pladform.ru/player?pl=18079&type=html5&videoid=101968178 ok.ru/video/241899607259


---------------------- 52784    adv ----------


стразы горный хрусталь пришивные 4 мм 1 руб./шт.


---------------------- 52785    recipe ----------


Лодочки из слоеного теста с картофелем, мясом и огурчиками в домашних условиях!  Рецепт с фото: >>>  http://http://r1.0-ede.ru/lodochki-iz-sloenogo-testa-s-kartofelem-myasom-i-ogurchikami/ Больше вкусных рецептов в группе:
https://ok.ru/group/56838620905523 ok.ru/proshuk ok.ru/group/56838620905523


---------------------- 52786     ----------


Остaвляя шрaмы другим свои не излeчишь.


---------------------- 52787     ----------


хуже всего скучать не по человеку, а по эмоциям которые он дарил, улыбки, смех, вечные подколы по поводу и без, вот чего нам не хватает.


---------------------- 52788    phone adv ----------


Хлеб для с/х животным в картофельных мешках
180р-от 10

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
train_texts.text.str.count('январ|феврал|март|апрел|май|июн|июл|август|сентябр|октябр|ноябр|декабр' + 
                                  '|понеденьник|вторник|среда|среду|четверг|пятниц|суббота|субботу|воскресе') \
        .clip(upper=1).mean()

0.0892525644137821

In [15]:
679252 / train_texts.shape[0]

0.19914064139955368

In [7]:
train_texts.shape

(3410916, 4)

In [11]:
train_texts.objectId.unique().shape

(3352714,)

In [41]:
train_texts.head(100000).text.str.count('\.(mpg|mp4|mp3)').sum()

207

In [7]:
train_texts

,objectId,lang,text,preprocessed
0,11181946,ru,"Питкерния\r\n\r\nОчень интересное растение, пр...","[питкерн, очен, интересн, растен, произраста, ..."
1,12040268,Unknown,"Яхты, олигархи, проститутки: секс-охотница раз...","[яхт, олигарх, проститутк, секс, охотниц, разо..."
2,14050867,ru,"Кто-то гибнет в бою, подрывая себя гранатой, а...","[гибнет, бо, подрыв, гранат, ког, ведут, бо, в..."
3,17023591,ru,Отношения: когда происходит выбор? Святослав Р...,"[отношен, происход, выбор, святосла, райк, чит..."
4,18389833,Unknown,ok.ru/group/51094392012955 ok.ru/giflive ok.ru...,[]
5,24344214,ru,ФК «Ротор-Волгоград» - «Спартак-2» (Москва) 1:...,"[фк, ротор, волгоград, спартак, москв, гол, ап..."
6,24544853,ru,И. Анисимов - История взятая из жизни (стихи) ...,"[анисим, истор, взят, жизн, стих]"
7,24583002,ru,"Как бы выглядели химические элементы, если бы ...","[выглядел, химическ, элемент, людьм]"
8,26053581,ru,Очень странные дела происходят с Томом Харди в...,"[очен, стран, дел, происход, том, хард, перв, ..."
9,26853081,ru,ДЕРЕВЯННАЯ ЛЕСТНИЦА - из натурального дерева ...,"[деревя, лестниц, натуральн, дерев, остаёт, пр..."


In [14]:
train_texts[['objectId', 'lang']].groupby('objectId').agg({'lang': 'count'})['lang'].reset_index() \
    .sort_values(['lang'], ascending=[False])

,objectId,lang
31766,487941,277
42767,556010,255
6477,339044,185
41284,546672,106
2,1283,73
44768,568009,59
41877,550628,55
23134,437460,55
1,677,41
1343547,18229844,33


In [11]:
a = train_texts[train_texts.objectId == 19465260]

In [12]:
for i in range(a.shape[0]):
    print(a.iloc[i,:].text)
    print('----------------------------------------------')

Scorpions- "Is There Anybody There" 1979 TV 50955946725 https://youtu.be/R49lfk9Y2b0
----------------------------------------------


In [21]:
train_texts[train_texts.objectId == 1195330]

,objectId,lang,text,preprocessed
165288,1195330,Unknown,160руб. - КОЛЬЦО. ОСТАТКИ РАЗМЕРОВ:\n3мм: 18.8...,"[руб, кольц, остатк, размер, мм, мм, xuping, п..."
613512,1195330,Unknown,160руб. - КОЛЬЦО. ОСТАТКИ РАЗМЕРОВ:\n3мм: 18(1...,"[руб, кольц, остатк, размер, мм, мм, xuping, п..."
646510,1195330,Unknown,"150руб. - КОЛЬЦО ОБРУЧАЛЬНОЕ ""XUPING"":\n3мм: 1...","[руб, кольц, обручальн, xuping, мм, мм, xuping..."
1106002,1195330,Unknown,"150руб. - КОЛЬЦО ОБРУЧАЛЬНОЕ ""XUPING"":\n3мм: 1...","[руб, кольц, обручальн, xuping, мм, мм, xuping..."
1115746,1195330,Unknown,"150руб. - КОЛЬЦО ОБРУЧАЛЬНОЕ ""XUPING"":\n3мм: 1...","[руб, кольц, обручальн, xuping, мм, мм, xuping..."
1605411,1195330,ru,"150руб. - КОЛЬЦО ОБРУЧАЛЬНОЕ ""XUPING"":\n3мм: 1...","[руб, кольц, обручальн, xuping, мм, мм, xuping..."
1648239,1195330,ru,"150руб. - КОЛЬЦО ОБРУЧАЛЬНОЕ ""XUPING"":\n3мм: 1...","[руб, кольц, обручальн, xuping, мм, мм, xuping..."
1691453,1195330,Unknown,"150руб. - КОЛЬЦО ОБРУЧАЛЬНОЕ ""XUPING"":\n3мм: 1...","[руб, кольц, обручальн, xuping, мм, мм, xuping..."
1708836,1195330,Unknown,"150руб. - КОЛЬЦО ОБРУЧАЛЬНОЕ ""XUPING"":\n3мм: 1...","[руб, кольц, обручальн, xuping, мм, мм, xuping..."
1757533,1195330,ru,"150руб. - КОЛЬЦО ОБРУЧАЛЬНОЕ ""XUPING"":\n3мм: 1...","[руб, кольц, обручальн, xuping, мм, мм, xuping..."


In [9]:
train_texts[train_texts.objectId == 24762087]

,objectId,lang,text,preprocessed
222919,24762087,ru,"Православный † календарь. Вторник, 27 марта, 2...","[православн, календар, вторник, март, март, ст..."
323206,24762087,ru,"Православный † календарь. Четверг, 29 марта, 2...","[православн, календар, четверг, март, март, ст..."
364810,24762087,ru,"Православный † календарь. Пятница, 23 марта, ...","[православн, календар, пятниц, март, март, ст,..."
507452,24762087,ru,"Православный † календарь. Воскресенье, 25 мар...","[православн, календар, воскресен, март, март, ..."
533885,24762087,ru,"Православный † календарь. Среда, 28 марта, 201...","[православн, календар, сред, март, март, ст, с..."
1082010,24762087,ru,"Православный † календарь. Суббота, 17 марта, 2...","[православн, календар, суббот, март, г, блгв, ..."
1140628,24762087,ru,"Православный † календарь. Воскресенье, 18 март...","[православн, календар, воскресен, март, март, ..."
1813732,24762087,ru,"Православный † календарь. Среда, 21 марта, 201...","[православн, календар, сред, март, г, икон, бо..."
2152473,24762087,ru,СРОЧНЫЙ СБОР СРЕДСТВ НА ВОССТАНОВЛЕНИЕ СПАСО -...,"[срочн, сбор, средств, восстановлен, спас, пре..."
2445603,24762087,ru,"Православный † календарь. Суббота, 24 марта, ...","[православн, календар, суббот, март, г, похва,..."
